In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import polars as pl
import gc

# Data Preparing

In [2]:
# Parquet 파일을 읽기
# polar를 활용하여 3번째 까지만 읽어옴
'''
for i in range(0,11,1):
    df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')
    globals()[f'df_{i + 1}'] = df

df_total = pl.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])
df_total.shape
'''

"\nfor i in range(0,11,1):\n    df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')\n    globals()[f'df_{i + 1}'] = df\n\ndf_total = pl.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])\ndf_total.shape\n"

In [3]:
# build datareading model
def read_data(num_f, num_s = 0):
    dfs = []
    for i in range(num_s,num_f):
        df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')
        dfs.append(df)

    df_total = pl.concat(dfs)
    return df_total

In [4]:
df_early_model = read_data(2)
df_mid_model = read_data(6, num_s = 4)
df_late_model = read_data(10, num_s = 8)

In [5]:
# feature grouping by Tag_n CSV, by distance = 2
# But just before, we gotta do scale the data!


g1 = [44,55,7,60,52,41,49]
g2 = [43, 54, 46, 57, 42, 53]
g3 = [6,59,37,38,5,8]
g4 = [47,58,39,50]
g5 = [40,48,51,45,56]
g6 = [0,1,32,33]
g7 = [2,34,3,4,35,36]
g8 = [20,31,30,29,28,27,26,25,24,23,21,22]
g9 = [70,12,67]
g10 = [71,13,68]
g11 = [72,14,69]
g12 = [73,74,75,76,77,78]
g13 = [17,64,15,62,16,63]
g14 = [61,11,9,10]
g15 = [18,65,19,66]


# Functions - handling data

In [6]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.mixture import GaussianMixture

def date_and_resp(df_tmp_total):
    # responder들 가져요기
    df_tmp_resp = df_tmp_total[:,83:92]

    # 첫 번째 행의 date_id 가져오기
    date_id_value = df_tmp_total["date_id"]

    # date_id 값을 새로운 열로 추가
    df_tmp_resp = df_tmp_resp.with_columns(
        pl.lit(date_id_value).alias("date_id")  # date_id 열 추가
    )

    return df_tmp_resp

def date_and_feature(df_tmp_total):
    # responder들 가져요기
    df_tmp_feat = df_tmp_total[:,4:83]

    # 첫 번째 행의 date_id 가져오기
    date_id_value = df_tmp_total["date_id"]  

    # date_id 값을 새로운 열로 추가
    df_tmp_feat = df_tmp_feat.with_columns(
        pl.lit(date_id_value).alias("date_id")  # date_id 열 추가
    )

    return df_tmp_feat

def sample_df_small(df):
    df_sampled = df.sample(n=2000000, with_replacement=False)
    return df_sampled

In [7]:
test_data = date_and_feature(df_early_model).drop('date_id')

# Functions - Clustering
- with no scaling

In [13]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

def kmeans(data, nclusters = 5, transpose = 'y'):

    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)
    
    if transpose == 'y':
        data_imputed.transpose()
    else:
        pass

    # K-프로토타입스 모델 생성
    n_clusters = 5  # 클러스터 개수 (그룹 개수)
    kmeans = KMeans(n_clusters=nclusters, random_state=42)

    # 클러스터링 수행
    clusters = kmeans.fit_predict(data_imputed)

    # 클러스터링 결과
    data = data.with_columns(pl.Series(name='Cluster', values=clusters))

    return data


In [16]:
kmeans_clustered_dat = kmeans(test_data)

c:\Users\campus3S031\Desktop\iMBank\iM_DiGital_Banker_academy\venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: ['feature_21' 'feature_26' 'feature_27' 'feature_31']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [15]:
kmeans_clustered_dat

feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,Cluster
f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32
null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,null,0.28118,0.269163,0.349028,…,-0.181716,null,null,null,0.564021,2.088506,0.832022,null,0.204797,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,2
null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,null,-1.512286,-1.414357,-1.823322,…,null,null,null,null,-10.835207,-0.002704,-0.621836,null,1.172836,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2
null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,null,-0.320921,-0.95809,-2.436589,…,null,null,null,null,-1.420632,-3.515137,-4.67776,null,0.535897,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,2
null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,null,0.544138,-0.087091,-1.500147,…,null,null,null,null,0.382074,2.669135,0.611711,null,2.413415,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,4
null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,null,-0.08842,-0.995003,-2.635336,…,null,null,null,null,-2.0146,-2.321076,-3.711265,null,1.253902,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.3

In [14]:
kmeans_clustered_dat['Cluster']

Cluster
i32
2
2
2
4
0
…
2
1
2
